# Initialization of libraries

In [34]:
# %matplotlib inline
import os
import sys
# import matplotlib

import numpy as np
import pandas as pd
import time
# import matplotlib.pyplot as plt
# import networkx as nx

from datetime import datetime
from collections import defaultdict
from sktensor import dtensor, cp_als

# matplotlib.style.use('ggplot')

# Manufacturing company emails

### Importing the dataset

In [2]:
# Looking for the dataset
if not os.path.isdir('radoslaw_email'):
    print 'The dataset has not be found'
else:
    # Loading the dataset
    tab = pd.read_csv('radoslaw_email/out.radoslaw_email_email', sep=' ', skiprows=2, header=None, index_col=False)

In [3]:
# Display
tab.head()

,0,1,2,3,4
0,1,2,NaN,1,1262454010
1,1,3,NaN,1,1262454010
2,1,4,NaN,1,1262454010
3,1,5,NaN,1,1262454010
4,1,6,NaN,1,1262454010


In [11]:
# Keeping desired columns and naming them
tab.drop(2, axis=1, inplace=True)
tab.columns = ['sender', 'receiver', 'weight', 'datetime']

# Date Time format
tab.datetime = pd.to_datetime(tab.datetime,unit='s')

ValueError: labels [2] not contained in axis

In [9]:
# Display
tab.drop('weight', axis=1, inplace=True)
tab.head()

,sender,receiver,datetime
0,1,2,2010-01-02 17:40:10
1,1,3,2010-01-02 17:40:10
2,1,4,2010-01-02 17:40:10
3,1,5,2010-01-02 17:40:10
4,1,6,2010-01-02 17:40:10


In [12]:
tab.datetime = pd.to_datetime(tab.datetime,unit='s')

In [13]:
tab.head()

,sender,receiver,datetime
0,1,2,2010-01-02 17:40:10
1,1,3,2010-01-02 17:40:10
2,1,4,2010-01-02 17:40:10
3,1,5,2010-01-02 17:40:10
4,1,6,2010-01-02 17:40:10


In [29]:
tab['month'] = tab.datetime.dt.month

In [68]:
tab.head()

,sender,receiver,datetime,month
0,1,2,2010-01-02 17:40:10,1
1,1,3,2010-01-02 17:40:10,1
2,1,4,2010-01-02 17:40:10,1
3,1,5,2010-01-02 17:40:10,1
4,1,6,2010-01-02 17:40:10,1


### Quick Data Analysis

In [56]:
print '%d mails exchanged between %d employees from %s to %s' %(tab.shape[0],
                                                                len(pd.unique(pd.concat([tab.sender,tab.receiver]))),
                                                                str(min(tab.datetime)),
                                                                str(max(tab.datetime))) 

82927 mails exchanged between 167 employees from 2010-01-02 17:40:10 to 2010-09-30 22:08:12


In [57]:
# tab.datetime.groupby(tab.datetime.dt.week).count().plot(
#     title='Temporal Distribution per week',
#     kind='bar',
#     figsize=(16, 10),
#     color='#348ABD',
#     alpha=0.6,
#     lw='1',
#     edgecolor='#348ABD',
#     grid=True)

## Creating a tensor

### Listing all the unique values of the dataset

In [70]:
X = tab.iloc[:, [0, 1, 3]]

i_list = pd.unique(X.iloc[:, 0])
j_list = pd.unique(X.iloc[:, 1])
k_list = pd.unique(X.iloc[:, 2])

i_list.sort()
j_list.sort()
k_list.sort()

In [71]:
i_ = list(tab.sender)
j_ = list(tab.receiver)
i_.sort()
j_.sort()

In [72]:
users = pd.unique(i_ + j_)

In [73]:
users.sort()
users

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167])

In [74]:
len(users)

167

### Creating and filling the tensor

In [75]:
T = np.zeros((len(users), len(users), len(k_list)))
T = dtensor(T)

In [76]:
# Going through the dataframe
t_temp = time.time()

for i, j, k in zip(X.sender, X.receiver, X.month):
    # Finding the corresponding index in the tensor
    i_ind = [i_ind for i_ind, x in enumerate(i_list) if x == i][0]
    j_ind = [j_ind for j_ind, x in enumerate(j_list) if x == j][0]
    k_ind = [k_ind for k_ind, x in enumerate(k_list) if x == k][0]
    
    # Modifying the tensor value for the tuple (i_ind, j_ind, k_ind)
    T[i_ind, j_ind, k_ind] += 1

print time.time()-t_temp    

33.2431950569


In [77]:
# Logarithmic Transformation
nonz = T.nonzero()
for ind in range(len(nonz[0])):
    i_ind = nonz[0][ind]
    j_ind = nonz[1][ind] 
    k_ind = nonz[2][ind]
    
    T[i_ind, j_ind, k_ind] = 1 + np.log(T[i_ind, j_ind, k_ind]) 

In [78]:
T.shape

(167, 167, 9)